# **TP Intelligence Artificielle - SAT et CSP**

# **Partie 0 : Visualisation**

Voici une fonction pour visualiser les sudokus.

In [1]:
from IPython.display import HTML


def visualize_sudoku(grid):

    html = "<table>"

    for i in range(9):

        html += "<tr>"

        for j in range(9):

            value = grid[i][j]

            cell_style = "width: 30px; height: 30px; text-align: center; font-size: 20px; border: 1px solid black;"

            if i % 3 == 0 and i != 0:  # Horizontal lines

                cell_style += "border-top: 5px solid black;"

            if j % 3 == 0 and j != 0:  # Vertical lines

                cell_style += "border-left: 5px solid black;"

            if value == 0:

                html += f"<td style='background-color: white; {cell_style}'> </td>"  # Case vide
            else:

                html += f"<td style='background-color: rgb(226, 114, 91); {cell_style}'>{value}</td>"

        html += "</tr>"

    html += "</table>"

    display(HTML(html))

### **Exercice 1**
Écrivez un Sudoku sous forme d’une liste de listes (9 listes de 9 élements chacune où la valeur 0 represente une caise vide), puis utilisez la fonction visualize_sudoku pour l’afficher.

In [2]:
grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 9, 7],
]

visualize_sudoku(grid)

5,3,,,7,,,,
6,,,1,9,5,,,
,9,8,,,,,6,
8,,,,6,,,,3
4,,,8,,3,,,1
7,,,,2,,,,6
,6,,,,,2,8,
,,,4,1,9,,,5
,,,,8,,,9,7


# **Partie 1 : Modélisation**

Nous allons modéliser le Sudoku comme un problème SAT et CSP, puis utiliser les méthodes propres à chaque modélisation pour la résolution.
Pour cela, nous utiliserons deux bibliothèques : **python-sat** et **python-constraint**.

Exécutez le bloc de code suivant pour les installer.
Pour les installer sur vos PC, il suffit d’exécuter les mêmes lignes (sans le '!' au début) dans une console de commandes (CMD).



In [3]:
# Installation des libraries
#!pip install python-sat
#!pip install python-constraint

## **1.1 Modélisation SAT**

Pour modéliser le Sudoku comme un problème SAT, nous utiliserons la méthode CNF de la bibliothèque pysat.formula.
CNF permet de définir les clauses logiques représentant les contraintes du problème.
Pour les variables, nous utiliserons un dictionnaire.

In [4]:
from pysat.formula import CNF


class Sudoku_SAT:
    def __init__(self, grid):
        self.grid = grid

        self.cnf = CNF()  # Crée un objet CNF pour stocker les clauses
        self.var_map = (
            {}
        )  # Dictionnaire pour mapper (ligne, colonne, valeur) à un ID de variable SAT

        # 1. Créer les variables SAT
        var_count = 1
        for i in range(9):
            for j in range(9):

                for k in range(1, 10):
                    self.var_map[(i, j, k)] = var_count
                    var_count += 1

        # 2. Créer les clauses SAT

        # a. Chaque case doit contenir au moins une valeur
        for i in range(9):
            for j in range(9):

                clause = [self.var_map[(i, j, k)] for k in range(1, 10)]
                self.cnf.append(clause)

        # b. Chaque case ne peut contenir qu'une seule valeur
        for i in range(9):
            for j in range(9):

                for k1 in range(1, 10):
                    for k2 in range(k1 + 1, 10):
                        clause = [-self.var_map[(i, j, k1)], -self.var_map[(i, j, k2)]]
                        self.cnf.append(clause)

        # c. Chaque ligne doit contenir tous les chiffres de 1 à 9
        for i in range(9):

            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for j in range(9)]
                self.cnf.append(clause)

        # d. Chaque colonne doit contenir tous les chiffres de 1 à 9
        for j in range(9):

            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for i in range(9)]
                self.cnf.append(clause)

        # e. Chaque sous-grille 3x3 doit contenir tous les chiffres de 1 à 9
        for subgrid_row in range(3):
            for subgrid_col in range(3):

                for k in range(1, 10):
                    clause = [
                        self.var_map[(subgrid_row * 3 + i, subgrid_col * 3 + j, k)]
                        for i in range(3)
                        for j in range(3)
                    ]
                    self.cnf.append(clause)

        # f. Contraintes initiales de la grille
        for i in range(9):
            for j in range(9):

                if self.grid[i][j] != 0:
                    self.cnf.append([self.var_map[(i, j, self.grid[i][j])]])

### **Exercice 2**
Créez une fonction qui prend une grille de Sudoku en entrée et utilise le solveur Glucose3 de la bibliothèque pysat.solver pour résoudre le jeu (pensez à bien importer Glucosa3). Votre fonction doit :

1. Créer un objet sudoku_sat pour modéliser le problème.
2. Ajouter au solveur les contraintes du Sudoku sous forme de clauses logiques.
3. Vérifier si le Sudoku est résoluble en utilisant le solveur SAT.
4. Trouver une solution si le Sudoku est résoluble.
5. Interpréter la solution retournée par le solveur pour remplir la grille :
  * La solution sera une liste de taille 9 × 9 × 10, contenant des valeurs positives et négatives.
  * Une valeur positive signifie que la variable (i, j, k) est True, ce qui indique que la case (i, j) doit contenir la valeur k.

In [5]:
from pysat.solvers import Glucose3


def solve_sudoku_sat(grid: list[list[int]]) -> list[list[int]]:
    sudoku_sat = Sudoku_SAT(grid)
    g3 = Glucose3()
    g3.append_formula(sudoku_sat.cnf)
    if g3.solve():
        solution = g3.get_model()

        grid = [[0 for _ in range(9)] for _ in range(9)]

        var_count = 0
        for i in range(9):
            for j in range(9):
                for k in range(1, 10):
                    if solution[var_count] > 0:
                        grid[i][j] = k
                    var_count += 1
        return grid
    else:
        return "Le Sudoku n'est pas résoluble"


solve_sudoku_sat(grid)

"Le Sudoku n'est pas résoluble"

### **Exercice 3**
Utilisez votre fonction pour resoudre le sudoku ci-dessous:

    [0, 0, 0, 0, 7, 0, 0, 0, 0]
    [6, 0, 0, 1, 9, 5, 0, 0, 0]
    [0, 9, 8, 0, 0, 0, 0, 6, 0]
    [8, 0, 0, 0, 6, 0, 0, 0, 3]
    [4, 0, 0, 8, 0, 3, 0, 0, 1]
    [7, 0, 0, 0, 2, 0, 0, 0, 6]
    [0, 6, 0, 0, 0, 0, 2, 8, 0]
    [0, 0, 0, 4, 1, 9, 0, 0, 5]
    [0, 0, 0, 0, 8, 0, 0, 7, 9]

In [6]:
grid = [
    [0, 0, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

visualize_sudoku(solve_sudoku_sat(grid))

5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


## **1.2 Modélisation CSP**

### **Exercice 4**

Créez une fonction solve_sudoku_csp qui prend une grille de Sudoku en entrée, la modélise comme un problème CSP et la résout. Votre fonction doit :
1. Créer une instance du problème CSP
  * Utilisez la classe Problem de la bibliothèque python-constraint pour représenter le Sudoku.
2. Définir les variables du problème
  * Utilisez la méthode addVariable pour ajouter les variables au problème, sachant que :
    * Chaque case de la grille est une variable identifiée par un tuple (ligne, colonne).
    * Le domaine de chaque variable correspond aux valeurs possibles (de 1 à 9).
    * Si une case contient déjà un chiffre (non nul), son domaine est restreint à cette valeur unique.
3. Définir les contraintes du Sudoku (les valeurs doivent être uniques dans chaque ligne, colonne et région 3×3). Pour cela, utilisez la méthode addConstraint avec AllDifferentConstraint() pour assurer ces contraintes.
4. Résoudre le problème. Pour cela utilisez la méthode getSolutions() pour obtenir toutes les solutions possibles.
5. Gérer les solutions
  * Si des solutions existent, les stocker dans une liste.
  * Sinon, indiquer qu’aucune solution n’a été trouvée.

In [7]:
from constraint import Problem, AllDifferentConstraint


def solve_sudoku_csp(grid: list[list[int]]) -> list[list[int]]:
    problem = Problem()

    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0:
                problem.addVariable((i, j), range(1, 10))
            else:
                problem.addVariable((i, j), [grid[i][j]])

    for i in range(9):
        problem.addConstraint(
            AllDifferentConstraint(), [(i, j) for j in range(9)]
        )  # row
        problem.addConstraint(
            AllDifferentConstraint(), [(j, i) for j in range(9)]
        )  # colomn

    for i in [0, 3, 6]:
        for j in [0, 3, 6]:
            problem.addConstraint(
                AllDifferentConstraint(),
                [(a + i, b + j) for a in range(3) for b in range(3)],
            )

    solution = problem.getSolution()
    for k, v in solution.items():
        grid[k[0]][k[1]] = v

    return grid

### **Exercice 5**
Utilisez votre fonction pour resoudre le jeu suivant.

In [8]:
# Grille de Sudoku (0 représente une case vide)
grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

In [9]:
visualize_sudoku(solve_sudoku_csp(grid))

5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


# **Partie 2 : Comparaison CSP vs SAT**

Nous allons comparer les deux solveurs sur 10 sudokus differents. Pour cela, voici 10 grilles.

In [10]:
grid1 = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

In [11]:
grid2 = [
    [0, 0, 3, 0, 2, 0, 6, 0, 0],
    [9, 0, 0, 3, 0, 5, 0, 0, 1],
    [0, 0, 1, 8, 0, 6, 4, 0, 0],
    [0, 0, 8, 1, 0, 2, 9, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 8],
    [0, 0, 6, 7, 0, 8, 2, 0, 0],
    [0, 0, 2, 6, 0, 9, 5, 0, 0],
    [8, 0, 0, 2, 0, 3, 0, 0, 9],
    [0, 0, 5, 0, 1, 0, 3, 0, 0],
]

In [12]:
grid3 = [
    [0, 2, 0, 6, 0, 8, 0, 0, 0],
    [5, 8, 0, 0, 0, 9, 7, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [3, 7, 0, 0, 0, 0, 5, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 8, 0, 0, 0, 0, 1, 3],
    [0, 0, 0, 0, 2, 0, 0, 0, 0],
    [0, 0, 9, 8, 0, 0, 0, 3, 6],
    [0, 0, 0, 3, 0, 6, 0, 9, 0],
]

In [13]:
grid4 = [
    [0, 0, 5, 3, 0, 0, 0, 0, 0],
    [8, 0, 0, 0, 0, 0, 0, 2, 0],
    [0, 7, 0, 0, 1, 0, 5, 0, 0],
    [4, 0, 0, 0, 0, 5, 3, 0, 0],
    [0, 1, 0, 0, 7, 0, 0, 0, 6],
    [0, 0, 3, 2, 0, 0, 0, 8, 0],
    [0, 6, 0, 5, 0, 0, 0, 0, 9],
    [0, 0, 4, 0, 0, 0, 0, 3, 0],
    [0, 0, 0, 0, 0, 9, 7, 0, 0],
]

In [14]:
grid5 = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0],
]

In [15]:
grid6 = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0],
]

In [16]:
grid7 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0],
]

In [17]:
grid8 = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 8, 5],
    [0, 0, 1, 0, 2, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 7, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 1, 0, 0],
    [0, 9, 0, 0, 0, 0, 0, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 7, 3],
    [0, 0, 2, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 9],
]

In [18]:
grid9 = [
    [0, 0, 0, 2, 6, 0, 7, 0, 1],
    [6, 8, 0, 0, 7, 0, 0, 9, 0],
    [1, 9, 0, 0, 0, 4, 5, 0, 0],
    [8, 2, 0, 1, 0, 0, 0, 4, 0],
    [0, 0, 4, 6, 0, 2, 9, 0, 0],
    [0, 5, 0, 0, 0, 3, 0, 2, 8],
    [0, 0, 9, 3, 0, 0, 0, 7, 4],
    [0, 4, 0, 0, 5, 0, 0, 3, 6],
    [7, 0, 3, 0, 1, 8, 0, 0, 0],
]

In [19]:
grid10 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0],
]

### **Exercice 6**

Créez une fonction comparer_solveurs qui prend comme entrée une liste de grilles, résout tous les jeux avec les deux solveurs, enregistre le temps d'exécution à chaque fois, et retourne les résultats sous forme de DataFrame.

Utilisez cette fonction pour résoudre tous les jeux et affichez ensuite les résultats.

In [20]:
import pandas as pd
from time import time


def comparer_solveurs(list_grids: list[list[list[int]]]) -> pd.DataFrame:
    df_results = pd.DataFrame(columns=["ID", "SAT", "CSP"]).astype(
        {"ID": str, "SAT": float, "CSP": float}
    )

    for i, grid in enumerate(list_grids):
        start = time()
        solve_sudoku_sat(grid)
        middle = time()
        solve_sudoku_csp(grid)
        end = time()
        df_results = pd.concat(
            [
                df_results,
                pd.DataFrame(
                    [[i + 1, middle - start, end - middle]],
                    columns=df_results.columns,
                ),
            ],
            ignore_index=True,
        )

    df_results["SAT"] = df_results["SAT"].round(2)
    df_results["CSP"] = df_results["CSP"].round(2)

    return df_results

In [21]:
from tabulate import tabulate

df_results = comparer_solveurs(
    [grid1, grid2, grid3, grid4, grid5, grid6, grid7, grid8, grid9, grid10]
)

print(
    tabulate(
        df_results,
        headers="keys",
        tablefmt="pretty",
        showindex=False,
        colalign=("right", "decimal", "decimal"),
    )
)

+----+------+------+
| ID |  SAT |  CSP |
+----+------+------+
|  1 | 0.02 | 0.01 |
|  2 | 0.02 | 0.01 |
|  3 | 0.1  | 0.25 |
|  4 | 0.29 | 0.09 |
|  5 | 0.38 | 0.53 |
|  6 | 0.28 | 0.2  |
|  7 | 0.28 | 0.5  |
|  8 | 0.34 | 4.14 |
|  9 | 0.01 | 0.0  |
| 10 | 0.32 | 0.53 |
+----+------+------+
